In [1]:
"""
Testing remote access to VIIRS active fire data
"""

import os, sys, time
import earthaccess as ea
import geopandas as gpd
import xarray as xr
import datetime as dt

from tqdm.notebook import tqdm
from urllib.parse import urlparse

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *
        
# Directories
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

# Output directories
dataraw = os.path.join(projdir,'data/spatial/raw/VIIRS/')
datamod = os.path.join(projdir,'data/spatial/mod/VIIRS/')

print("Ready !")

Ready !


In [2]:
# Load the fire dataset for the Southern Rockies
fires = gpd.read_file(os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen-obs.gpkg'))
fires = fires[fires['na_l3name'] == 'Southern Rockies']

# tidy the fire id and name columns
fires.rename(columns={'NIFC_ID': 'Fire_ID', 'NIFC_NAME': 'Fire_Name'}, inplace=True)
fires['obs_count'] = fires['obs_count'].fillna(0).astype(int) # fill NaN as 0 obs.
fires = fires[fires['obs_count'] >= 10]

# tidy the date columns
fires['DISCOVERY_DATE'] = pd.to_datetime(fires['DISCOVERY_DATE']).dt.date
fires['WF_CESSATION_DATE'] = pd.to_datetime(fires['WF_CESSATION_DATE']).dt.date

print(f"Available attributes: \n{fires.columns}")
print(f"\nThere are [{len(fires)}] fires.")

Available attributes: 
Index(['Fire_ID', 'Fire_Name', 'NIFC_ACRES', 'FINAL_ACRES', 'pct_aspen',
       'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'WF_CESSATION_DATE', 'WF_CESSATION_DOY',
       'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL', 'STR_THREATENED_MAX',
       'EVACUATION_REPORTED', 'PEAK_EVACUATIONS', 'WF_PEAK_AERIAL',
       'WF_PEAK_PERSONNEL', 'na_l3name', 'first_obs_date', 'last_obs_date',
       'obs_count', 'geometry'],
      dtype='object')

There are [50] fires.


In [4]:
# Grab a test fire (Cameron Peak)

fire = fires[fires['Fire_Name'] == '416']

buffer=1000
coords, extent = get_coords(fire, buffer)
print(f"Bounding coordinates: {coords}")

start_date = fire['first_obs_date'].iloc[0]
last_date = fire['last_obs_date'].iloc[0]
date_range = (start_date, last_date)
print(f"Date range for EA search request: {date_range}")

Bounding coordinates: [(-108.0104287321153, 37.37737010107216), (-107.79428582032635, 37.37737010107216), (-107.79428582032635, 37.60899484292132), (-108.0104287321153, 37.60899484292132), (-108.0104287321153, 37.37737010107216)]
Date range for EA search request: (Timestamp('2018-06-01 00:00:00'), Timestamp('2018-07-06 00:00:00'))


In [8]:
date_range = ('2018-06-01', '2018-06-04')

In [ ]:
# Search query with earthaccess

In [9]:
query = ea.search_data(
    short_name=['VJ114IMG', 'VNP14IMG'], 
    polygon=coords,
    temporal=date_range, 
    cloud_hosted=True,
    count=-1
)

Granules found: 17


In [10]:
# open the fileset remotely
fileset = ea.open(query)

Opening 17 granules, approx size: 0.04 GB


QUEUEING TASKS | :   0%|          | 0/17 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/17 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/17 [00:00<?, ?it/s]

In [11]:
print(fileset[0])

<File-like object HTTPFileSystem, https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/VNP14IMG.002/VNP14IMG.A2018152.0824.002.2024080110710/VNP14IMG.A2018152.0824.002.2024080110710.nc>


In [ ]:
# # isolate the fire swaths from the geolocation swaths
# vj114_files = [f for f in fileset if 'VJ114IMG' in str(f)]
# print(f"Processing a total of {len(vj114_files)} fire swaths.")

In [12]:
# load the lookup table for pixel sizes
lut = pd.read_csv(os.path.join(projdir,'data/tabular/raw/pix_size_lut.csv'))

In [14]:
t0 = time.time()

granule_dfs = []
for fp in tqdm(fileset[0:1], desc="Processing granules"):
    t1 = time.time()
    df = pd.DataFrame() # to store the active fire data
    with xr.open_dataset(fp, phony_dims='access') as swath:
        print(swath.attrs)
        print(swath.variables)
        # get the granule ID and associated geolocation swath
        granule_id = swath.LocalGranuleID
        geo_id = swath.VNP03IMG
        
        # Check for fire pixels in the specified region
        lonfp = swath.variables['FP_longitude'][:] # fire pixel longitude
        latfp = swath.variables['FP_latitude'][:] # fire pixel latitude
        fire_scene = ((lonfp > extent[0]) & (lonfp < extent[1]) & 
                      (latfp > extent[2]) & (latfp < extent[3]))
        
        if not fire_scene.any():  # Check for any fire pixels in region
            continue # skip if no fire pixels in region

        del fire_scene
        
        # granule attributes
        daynight = swath.DayNightFlag #string Day or Night

        # variables
        fire = swath['fire mask'] # the fire mask
        frp = swath.variables['FP_power'][:] # fire radiative power
        t4 = swath.variables['FP_T4'][:] # I04 brightness temp (kelvins)
        t5 = swath.variables['FP_T5'][:] # I05 brightness temp (kelvins)
        sample = swath.variables['FP_sample'][:]
        line = swath.variables['FP_line'][:]

        fire_mask = swath['fire mask'][line, sample].values
        
    # gather information from file name
    timestamp = granule_id.split('.')[1:3]
    year = timestamp[0][1:5]
    day = timestamp[0][5:8]
    acqtime = timestamp[1]
    acqdate = dt.datetime.strptime(year+day, '%Y%j').strftime('%-m/%-d/%Y')
    
    df['longitude'] = lonfp
    df['latitude'] = latfp
    df['j'] = sample #sample number for pixel size lookup
    df['fire_mask'] = fire_mask
    df['confidence'] = pd.Categorical( df.fire_mask)
    df.confidence = df.confidence.replace(
        {0:'x', 1:'x', 2:'x', 3:'x', 4:'x', 5:'x', 6:'x', 7:'l', 8:'n', 9:'h'})
    df['frp'] = frp
    df['acq_date'] = acqdate
    df['acq_time'] = acqtime
    df['daynight'] = swath.DayNightFlag
    df['satellite'] = swath.PlatformShortName
    df['short_name'] = swath.ShortName
    df['granule_id'] = granule_id
    df['geo_id'] = geo_id

    df = pd.merge(df, lut, left_on='j', right_on='sample', how='left')
    df.drop(columns=['j'], inplace=True)

    granule_dfs.append(df)

afds = pd.concat(granule_dfs, ignore_index=True)
print(len(afds))

t2 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t2:.2f} minutes.\n")
print("\n~~~~~~~~~~\n")
print("Done!")

Processing granules:   0%|          | 0/1 [00:00<?, ?it/s]

{'AlgorithmType': 'OPS', 'AlgorithmVersion': 'NPP_PR14IMG 3.1.6', 'CloudPix': 7508017, 'Conventions': 'CF-1.6', 'Day/Night/Both': 'Night', 'DayNightFlag': 'Night', 'DayPix': 0, 'EastBoundingCoordinate': -75.882835, 'EndTime': '2018-06-01 08:30:00.000', 'ExecutableCreationDate': 'Oct 31 2023', 'ExecutableCreationTime': '17:46:00', 'FirePix': 836, 'GRingPointLatitude': array([24.5584, 29.2576, 50.0588, 43.9237]), 'GRingPointLongitude': array([ -86.7116, -117.219 , -115.892 ,  -75.8828]), 'GRingPointSequenceNo': array([1, 2, 3, 4], dtype=int32), 'GlintPix': 0, 'InputPointer': '/MODAPSops4/archive/f20247/running/VNP_L1bglu/59465911/VNP02CCIMG.A2018152.0824.002.2022280024843.nc, /MODAPSops4/archive/f20247/running/VNP_L1bglu/59465911/VNP03IMG.A2018152.0824.002.2021082143054.nc, /MODAPSops4/archive/f20247/running/VNP_L1bglu/59465911/VNP02CCMOD.A2018152.0824.002.2022280024843.nc, /MODAPSops4/archive/f20247/running/VNP_L1bglu/59465911/VNP02GDC.A2018152.0824.002.2021082160332.nc', 'LandPix': 0, 

ValueError: No objects to concatenate

In [ ]:
afds.head()

In [ ]:
afds.to_csv(os.path.join(datamod,'afds_testing_416.csv'))